# Quantum Approximate Optimization Algorithm (QAOA)
#### ColibriTD 

Travail original de Hamza JAFFALI - hamza.jaffali@colibritd.com, Henri DE BOUTRAY - henri.de.boutray@colibritd.com et Julien CALISTO - julien.calisto@colibritd.com, adapté pour l'OA Quantique à l'ECE par Timothé PRESLES - timothe.presles@colibritd.com.

(envoyez moi quand même votre TP à mon adresse mail ECE)

In [21]:
# # # Permet d'installer le language. Ne doit être executé qu'une seule fois.
# NORMALEMENT, tout le nécessaire est import avec MPQP, mais si certaines bibliothèques ne le sont pas, j'en dirai deux mots à Julien CALISTO

!pip install mpqp --upgrade

In [17]:
from mpqp.tools import Matrix
from mpqp.measures import I, Z, Y, X, PauliString

## Préambule 

Dans l'import précédent, les fonctions "I, Z, Y et X" sont importées. CE NE SONT PAS DES PORTES QUANTIQUES mais des observables !!!
C'est une distiction assez peu intuitive de prime abord que j'avais mentionné en parlant du quantum annealing. Ici, ces objets mathématiques sont utilisés pour "encoder" l'information dans le Hamiltonien, mais pas directement pour réaliser une opération (vous le verrez par la suite, mais on utilisera une exponentielle pour encoder nos portes quantiques). Bref, cela n'est pas fondamental pour la compréhension de ce TP, mais je devais vous le dire pour être tout à fait rigoureux. D'ailleurs, dans la version à venir de MPQP (probablement la prochaine), ces observables seront notés 'pI, pX, pY et pZ' pour éviter la confusion !

## L'Algorithme Quantique Variationel QAOA : Quantum Approximate Optimization Algorithm

#### Rappelons le principe de l'algorithme

L'algorithme QAOA est un algorithme connu de la classe des algorithmes quantiques variationel (VQA en anglais, pour Variational Quantum Algorithm). Son bon fonctionnement nécessite 3 ingrédients principaux.

+ Une fonction de coût, encodant le problème d'optimisation que l'on souhaite résoudre, modélisée par un Hamiltonien $H_C$.
+ Un Ansatz décrivant l'architecture du circuit quantique utilisé pour amplifier les probabilités des solutions optimales, comprenant notre hamiltonien de coût $H_C$ un un Hamiltonien de "mixage" $H_M$. Pour rappel, on retrouve les Hamiltoniens dans l'équation de Schrödinger, et ce sont les mêmes opérateurs que ceux introduits lorsque nous avons parlé de quantum annealing / recuit quantique.
+ Un optimiseur classique pour chercher les paramètres optimaux de notre circuit quantique (de manière très similaire aux optimiseurs utilisés dans des réseaux de neurones.

Nous allons dans ce TP chercher à développer notre propre QAOA, en utilisant la bibliothèque de programmation quantique MPQP, développée par la startup ColibriTD. L'idée est de résoudre un problème simple: Trouver le minimum d'une fonction $C(x)$, définie comme:

$$ C(x) = 3x_0x_1 - 4x_0 - 2x_1 + 1 $$

La première étape est de définir un Hamiltonien $H_C$. On va chercher à le définir de telle sorte que son minimum d'énergie (ground state) corresponde au minimum de la fonction de coût. Cette fonction admettra deux variables $x_0$ et $x_1$. Ainsi, on utilisera un système à deux qubits $\ket{x}=\ket{x_0x_1}$ encodant une supperposition des quatre combinaisons binaires possibles.

Par la suite, nous utiliserons la relation suivante pour définir nos Hamiltoniens à partir de fonction mathématiques, comme notre fonction de coût par exemple:

$$ 
x_i = \frac{I^{\otimes n} - Z_i}{2} 
$$  
$$ 
\text{avec } ~~ Z_i = \underbrace{I \otimes \cdots \otimes I}_{i} \otimes Z \otimes \underbrace{I \otimes \cdots \otimes I}_{n-i-1} 
$$


> **_IMPORTANT :_** **Dans le cas de PauliString en MPQP**, le produit de Kronecker $ \otimes $ s'écrit "@".
> 
> Ne surtout pas confondre avec le produit matriel en numpy, qui pour deux matrices  $A$ et $B$ s'écrit de la même manière mais fait $A \times B$ au lieu de $A \otimes B$  !!!
> Ici, nous utiliserons uniquement @ dans le cadre des PauliString

**1) Ecrire une fonction python qui renvoie le Hamiltonien $H_C$ associé à la fonction de coût :**

In [ ]:
def hamiltonian_cost() -> PauliString:

    return ...

On définit le Hamiltonien de mélange $H_M$ comme une somme d'observables locales de Pauli $X_i$ tel que:
$$ 
H_M = \sum_{i=0}^{n-1} X_i 
$$ 
$$ 
\text{avec } ~~ X_i = \underbrace{I \otimes \cdots \otimes I}_{i} \otimes X \otimes \underbrace{I \otimes \cdots \otimes I}_{n-i-1} 
$$

**2) Ecrire une fonction python qui renvoie le Hamiltonien $H_M$ associé à l'opérateur de mélange :**

In [ ]:
def hamiltonian_mixer() -> PauliString:
    
    return ...

Une fois ces Hamiltoniens définis, il faut créer l'opérateur d'évolution qui sera répété plusieurs fois afin d'amplifier les probabilités de trouves les solutions de plus faible énergie i.e. minimisant notre fonction de coût. On rappelle ici que l'évolution temporelle décrite par un Hamiltonien indépendant du temps $H$ peut être représenté par un opérateur unitaire à 1 paramètre $U(\gamma) = e^{-i\gamma H}$. Cela implique que l'on doive calculer l'exponentielle d'une matrice, ce qui est permis par la fonction `expm` de la librairie `scipy.linalg`.

> **_Astuce :_** La fonction de MPQP PauliString.to_matrix() permet de convertir un objet Pauli string en une matrice associée.

In [ ]:
from scipy.linalg import expm

**3) Ecrire une fonction python qui renvoie $U_{C}(\gamma)$ associée au Hamiltonien $H_C$ et au paramètre $\gamma$ :**

In [ ]:
def unitary_matrix_hamiltonian_cost(gamma: float) -> Matrix:
    
    return ...

**4) Ecrire une fonction python qui renvoie $U_{M}(\beta)$ associée au Hamiltonien $H_M$ et au paramètre $\beta$ :**

In [ ]:
def unitary_matrix_hamiltonian_mixer(beta: float) -> Matrix:
    
    return ...

La prochaine étape est de construire le circuit quantique "paramétré" du QAOA, que l'on notera $U$, et qui dépend de $2d$ parametres $\{\gamma_1, \beta_1, \dots,\gamma_d, \beta_d\}$, avec $d$ la "profondeur" de notre Ansatz. Rappellons ici que le circuit du QAOA est composé d'une alternance d'opérateurs unitaires encodants l'évolution du système selon les Hamiltoniens $H_C$ et $H_M$ , avec un paramètre associé à chaque occurence de chaque opérateur.

En d'autres termes, on a:
$$U(\gamma_1, \beta_1, \dots,\gamma_d, \beta_d) = e^{-i\beta_d H_M}e^{-i\gamma_d H_C} \cdots e^{-i\beta_1 H_M}e^{-i\gamma_1 H_C}$$

**5) Ecrire une fonction python qui prend en entrée les paramètres de l'Ansatz et qui renvoie le circuit du QAOA en sortie:**

> **_Astuce :_** On doit créer des "CustomGate", qui se définissent comme nos "objets gate" dans les TPs précédents en Qiskit. Le language change, mais la logique reste strictement la même:
>+ Ce sont des objets que l'on doit "append" à notre circuit
>+ Leur premier paramètre est la matrice unitaire définissant l'action de la porte quantique
>+ Le second argument est la liste de qubits "targets" i.e. auxquels elles sont appliquées (a noter que si une porte à 1 qubits à plusieurs "targets", alors la même porte sera appliquée à l'ensemble de la liste de targets)

In [5]:
from typing import Optional

from mpqp.core.circuit import QCircuit
from mpqp.gates import Gate, CustomGate, UnitaryMatrix

def construct_qaoa_ansatz(parameters: list[float], depth: Optional[int]=None):
    if depth is None:
        if len(parameters) % 2 != 0:
            raise ValueError("parameters length must be even (2*depth).")
        depth = len(parameters) // 2

    if len(parameters) != 2 * depth:
        raise ValueError("Wrong parameters length vs depth.")

    gates: list[Gate] = []
     
    for d in range(depth):
        gamma = parameters[2 * d]
        beta  = parameters[2 * d + 1]

        # Rajouter à partir d'ici les portas quantiques paramétrées
        
        ...
    
    return QCircuit(gates)

Affichons le circuit avec quelques paramètres factices pour vérifier qu’il fonctionne correctement…


In [ ]:
construct_qaoa_ansatz([1,2,3,4,5,6,7,8]).display(warn=False)

Maintenant que nous avons la structure de notre circuit à optimiser, nous nous concentrons sur un élément important de notre algorithme : le calcul de la fonction de perte à optimiser. Ici, nous voulons minimiser la fonction de coût $C(x)$, ce qui revient à trouver l’état fondamental $\ket{\psi}$ du hamiltonien de coût $H_C$, lequel peut s’exprimer comme

$$
\ket{\psi} = \text{arg } \min_{\ket{x}} \ \langle x \mid H_C \mid x \rangle
$$

Ainsi, à chaque itération de l’algorithme, nous évaluerons la valeur d’espérance de $H_C$ par rapport à l’état produit par le circuit paramétré, et l’optimiseur mettra ensuite à jour les paramètres pour minimiser cette valeur. Nous proposons de calculer cette valeur d’espérance en utilisant le circuit généré par la fonction précédente ainsi que les autres fonctionnalités fournies. Vous pouvez utiliser `Observable`.

**6) Implémentez la fonction de perte qui calcule la fonction de coût à partir d’une liste de paramètres fournie en entrée:**

In [ ]:
from mpqp.core.instruction.measurement import Observable, ExpectationMeasure
from mpqp.execution import run, AvailableDevice

In [ ]:
def loss_function(parameters: list[float], device: AvailableDevice):
    
    # Récupérer le Hamiltonien de cout sous la forme d'une matrice pour créer l'observable
    cost_hamiltonian_matrix = hamiltonian_cost()
    observable: Observable = ...
    
    # Créer le circuit avec les fonction définies précédemment et l'observable
    circuit: QCircuit = ...
    circuit.add(ExpectationMeasure(observable))
    
    # Récupérer la valeur d'espérance (expectation value / exp_value) ) partir du résultat
    result = run(circuit, device)
    exp_value = ...
    
    return exp_value

Le dernier élément manquant de cet algorithme variationnel est l’optimiseur classique qui va optimiser ces paramètres et renvoyer l’état solution le plus probable. Nous définissons une fonction `solve_qaoa` qui prend comme paramètre une liste initiale de paramètres pour démarrer l’optimisation ainsi que la profondeur de l’ansatz. Elle appelle la fonction `minimize` du package `scipy.optimize`. Nous définissons une fonction de rappel (callback) pour suivre la valeur de la fonction de perte pendant l’optimisation. Vous pouvez ajuster les paramètres de l’optimiseur (méthode, tolérance, nombre d’itérations, etc.) jusqu’à ce que vous puissiez trouver la solution. Vous pouvez vous référer à la documentation suivante : https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html

*N'hésitez pas à tester avec différents paramètres d'optimiseur et différentes méthodes dans la suite de ce TP, et à regarder leur influence sur la qualité du résultat obtenu !*

La fonction *solve_qaoa* renvoie deux valeurs : le vecteur des meilleurs paramètres et la valeur minimale trouvée pour la fonction de coût.

In [ ]:
from typing import Any
from scipy.optimize import minimize

In [ ]:
def solve_qaoa(init_params: list[float], 
               method: str, 
               device: AvailableDevice,
               options: dict[str, Any],
               depth: Optional[int]=None, 
               ):

    def obj(params):
        return loss_function(params, device=device)
    
    def callback(xk): # Fonction cosmétique, renvoie la valeur de la loss à chaque itération
        print("current loss: ", obj(xk))

    opt = minimize(obj, init_params, method=method, options=options, callback=callback)
    return opt.x, opt.fun

Nous sommes maintenant prêts à tester l’algorithme ! La dernière étape consiste à initialiser l’ensemble des paramètres et à définir la profondeur de l’ansatz QAOA. Vous pouvez ensuite exécuter l’algorithme et vérifier si la fonction a réellement atteint son minimum.  

In [ ]:
from mpqp.execution import IBMDevice, ATOSDevice, AWSDevice, AZUREDevice, GOOGLEDevice

**7) Définissez les hyperparamètres l'optimiseur, vous pouvez notamment commencer par choisir une méthode (method) et un nombre d'itérations ( associé à la clé 'maxiter' dans dans le dictionnaire des options:**

In [ ]:
method = ... # TODO: tune
options = {"disp": True, "maxiter": ...} # TODO: tune

**8) Définissez les hyperparamètres du circuit quantique, en déterminant la profondeur (depth, nombre de répétition des paires d'opérateurs), les paramètres initiaux (pouvant être choisis aléatoirement ou non). On peut également choisir différents "devices" (ce qui est la force de MPQP), mais on restera sur la machine d'IBM simulée pour ce TP**

In [ ]:
depth = ...
init_params = ...
device = IBMDevice.AER_SIMULATOR # Littéralement celui utilisé dans nos TPs sur Qiskit

best_params, best_value = solve_qaoa(init_params, method, device, options, depth)
print(best_params, best_value)

L’algorithme produit un état pour lequel la solution est encodée dans l’état de base ayant la probabilité de mesure la plus élevée. Pour récupérer la solution la plus probable, nous devons échantillonner le circuit avec les meilleurs paramètres. La représentation binaire des états de base nous donne l’assignation des variables comme solution possible. Vous pouvez exécuter le snippet de code suivant et vérifier qu’il renvoie la bonne solution.


In [ ]:
from mpqp.measures import BasisMeasure

In [ ]:
qcircuit_best = construct_qaoa_ansatz(best_params)
qcircuit_best.add(BasisMeasure())
sol_res = run(qcircuit_best, device)
print(sol_res)

print("\nMost probable states are:")
for sample in sol_res.samples:
    if sample.probability > 0.09:
        s = sample.bin_str
        print(f"|{s}>", "{:.2f}%".format(100 * sample.probability), "--> Solution: {x_0 =",s[0], ", x_1 =",s[1]+"}")

Félicitations, vous avez implémenté un algorithme variationnel QAOA ! Pour aller plus loin, n'hésitez pas à complexifier la fonction de coût initiale (en rajoutant une variable et un terme où ces dernières se multiplient entre elles) et à refaire le TP avec. Vous observerez certainement que la profondeur de circuit nécessaire ainsi que le nombre d'itérations augmente légèrement.

#### Merci à tous pour votre sérieux, vos questions et votre attention ces deux derniers mois !